In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib as plt

In [2]:
from NRI.experiments.config import ExperimentConfig


experiment_config = ExperimentConfig(
    obs_len=12,
    pred_len=24,
    encoder_loss_weight=1.0,
    decoder_loss_weight=1.0,
    signal_loss_weight=1.0,
    n_epoch=20,
    batch_size=32,
    checkpoint_prefix='dnri',
    checkpoint_interval=10,
    use_cuda=True,
    mask_traffic_signal=False
)

In [3]:
from SinD.config import get_dataset_path
from SinD.dataset.io import get_dataset_records
from NRI.dataset.utils import split_dataset

dataset_path = get_dataset_path()
dataset_files = get_dataset_records(dataset_path)
train_files, valid_files, test_files = split_dataset(dataset_files, 0.7, 0.2)

load datasets

In [4]:
from NRI.dataset import SignalizedIntersectionDatasetForNRI, SignalizedIntersectionDatasetConfig

dataset_config = SignalizedIntersectionDatasetConfig(
    obs_len=experiment_config.obs_len,
    pred_len=experiment_config.pred_len,
    stride=15,
    encode_traffic_signals=True,
    padding_value=0.0
)

train_set = SignalizedIntersectionDatasetForNRI(dataset_config)
train_set.load_records(dataset_path, train_files, verbose=True)

valid_set = SignalizedIntersectionDatasetForNRI(dataset_config)
valid_set.load_records(dataset_path, valid_files, verbose=True)


load_records: 100%|██████████| 4/4 [00:42<00:00, 10.63s/it]


In [5]:
from NRI.models import DynamicNeuralRelationalInference
from NRI.experiments.main import train

model = DynamicNeuralRelationalInference(
    hid_dim=64,
    n_edges=4,
    dgvae=False
)

if experiment_config.use_cuda:
    model.cuda()

optimizer = torch.optim.Adam(model.parameters())

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

train(
    model,
    train_set=train_set,
    valid_set=valid_set,
    config=experiment_config,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler
)

[train] epoch 0:   0%|          | 0/341 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

Overall Loss: 24.984617171749026
Encoder KL Loss: 0.012981241496547483
Decoder NLL Loss: 23.97270452661596
Signal Cross-Entropy Loss: 0.9989312467686954


[valid] epoch 0:   0%|          | 0/95 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: Th

Signal Prediction Accuracy: 0.8040881156921387
Final Displacement Error: 5.993442038485878
Average Displacement Error: 2.8577335834503175


[train] epoch 1:   0%|          | 0/341 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: T

...

Overall Loss: -3.8111030730334203
Encoder KL Loss: 0.06377414024033504
Decoder NLL Loss: -4.505260211543952
Signal Cross-Entropy Loss: 0.6303829923927614


[valid] epoch 9:   0%|          | 0/95 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: Th

Signal Prediction Accuracy: 0.8186302185058594
Final Displacement Error: 7.130697024495978
Average Displacement Error: 2.7560066072564378


[train] epoch 10:   0%|          | 0/341 [00:00<?, ?it/s]/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(
/home/mattb/.local/lib/python3.12/site-packages/torch/nested/__init__.py:166: UserWarning: 

AssertionError: 

In [ ]:
import torch.distributions as td

corr = 9.6457e-01
sigma_x = 4.1535e-01
sigma_y = 1.0788e-01

cov = torch.tensor([
    [sigma_x ** 2, sigma_y * corr * sigma_x],
    [corr * sigma_x * sigma_y, sigma_y ** 2],
])

chol = torch.tensor([
    [sigma_x, 0.0],
    [corr * sigma_y, torch.sqrt(1 - torch.tensor(corr) ** 2) * sigma_y]
])

print(td.MultivariateNormal(
    loc=torch.tensor([-1.3988e-02,  1.6305e-02]),
    covariance_matrix=cov
).log_prob(torch.tensor([0.0, 0.0])))

print(td.MultivariateNormal(
    loc=torch.tensor([-1.3988e-02,  1.6305e-02]),
    scale_tril=chol
).log_prob(torch.tensor([0.0, 0.0])))